In [1]:
%run init.ipynb

matchzoo version 1.0
`ranking_task` initialized with metrics [normalized_discounted_cumulative_gain@3(0.0), normalized_discounted_cumulative_gain@5(0.0), mean_average_precision(0.0)]
data loading ...
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`


In [2]:
preprocessor = mz.models.MatchSRNN.get_default_preprocessor()

In [3]:
train_pack_processed = preprocessor.fit_transform(train_pack_raw)
dev_pack_processed = preprocessor.transform(dev_pack_raw)
test_pack_processed = preprocessor.transform(test_pack_raw)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 9513.80it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:03<00:00, 5600.48it/s]
Building Vocabulary from a datapack.: 100%|██████████| 418401/418401 [00:00<00:00, 2271254.89it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 7783.74it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:03<00:00, 5277.22it/s]
Processing length_right with len: 100%|██████████| 18841/18841 [00:00<00:00, 938705.01it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 122/122 [00:00<00:00, 10313.31it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 1115/1115 [00:00<0

In [4]:
preprocessor.context

{'filter_unit': <matchzoo.preprocessors.units.frequency_filter.FrequencyFilter at 0x7f68141f1208>,
 'vocab_unit': <matchzoo.preprocessors.units.vocabulary.Vocabulary at 0x7f6814509780>,
 'vocab_size': 30058,
 'embedding_input_dim': 30058}

In [5]:
glove_embedding = mz.datasets.embeddings.load_glove_embedding(dimension=100)
term_index = preprocessor.context['vocab_unit'].state['term_index']
embedding_matrix = glove_embedding.build_matrix(term_index)
l2_norm = np.sqrt((embedding_matrix * embedding_matrix).sum(axis=1))
embedding_matrix = embedding_matrix / l2_norm[:, np.newaxis]

In [6]:
trainset = mz.dataloader.Dataset(
    data_pack=train_pack_processed,
    mode='pair',
    num_dup=2,
    num_neg=1
)
testset = mz.dataloader.Dataset(
    data_pack=test_pack_processed
)

In [7]:
padding_callback = mz.models.ArcII.get_default_padding_callback()

trainloader = mz.dataloader.DataLoader(
    dataset=trainset,
    batch_size=20,
    stage='train',
    resample=True,
    sort=False,
    callback=padding_callback
)
testloader = mz.dataloader.DataLoader(
    dataset=testset,
    batch_size=20,
    stage='dev',
    callback=padding_callback
)

In [8]:
model = mz.models.MatchSRNN()

model.params['task'] = ranking_task
model.params['embedding'] = embedding_matrix
model.params['channels'] = 4
model.params['units'] = 10
model.params['dropout'] = 0.2
model.params['direction'] = 'lt'

model.build()

print(model)
print('Trainable params: ', sum(p.numel() for p in model.parameters() if p.requires_grad))

MatchSRNN(
  (embedding): Embedding(30058, 100)
  (dropout): Dropout(p=0.2, inplace=False)
  (matching_tensor_network): MatchingTensor()
  (spatial_gru_network): SpatialGRU(
    (_activation): Tanh()
    (_recurrent_activation): Sigmoid()
  )
  (out): Linear(in_features=10, out_features=1, bias=True)
)
Trainable params:  3048611


In [9]:
optimizer = torch.optim.Adadelta(model.parameters())

trainer = mz.trainers.Trainer(
    model=model,
    optimizer=optimizer,
    trainloader=trainloader,
    validloader=testloader,
    validate_interval=None,
    epochs=10
)

In [10]:
trainer.run()

[Iter-102 Loss-0.990]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4586 - normalized_discounted_cumulative_gain@5(0.0): 0.5171 - mean_average_precision(0.0): 0.4899



[Iter-204 Loss-0.735]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.544 - normalized_discounted_cumulative_gain@5(0.0): 0.6028 - mean_average_precision(0.0): 0.5636



[Iter-306 Loss-0.563]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5717 - normalized_discounted_cumulative_gain@5(0.0): 0.6379 - mean_average_precision(0.0): 0.5955



[Iter-408 Loss-0.412]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5598 - normalized_discounted_cumulative_gain@5(0.0): 0.6327 - mean_average_precision(0.0): 0.5853



[Iter-510 Loss-0.297]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5639 - normalized_discounted_cumulative_gain@5(0.0): 0.6361 - mean_average_precision(0.0): 0.5816



[Iter-612 Loss-0.192]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5394 - normalized_discounted_cumulative_gain@5(0.0): 0.6161 - mean_average_precision(0.0): 0.5709



[Iter-714 Loss-0.145]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5067 - normalized_discounted_cumulative_gain@5(0.0): 0.5728 - mean_average_precision(0.0): 0.5293



[Iter-816 Loss-0.105]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5573 - normalized_discounted_cumulative_gain@5(0.0): 0.6236 - mean_average_precision(0.0): 0.5822



[Iter-918 Loss-0.081]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5311 - normalized_discounted_cumulative_gain@5(0.0): 0.5964 - mean_average_precision(0.0): 0.5539



[Iter-1020 Loss-0.058]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5272 - normalized_discounted_cumulative_gain@5(0.0): 0.6042 - mean_average_precision(0.0): 0.5583

Cost time: 9307.04870057106s
